In [2]:
import wrangle_zillow

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = wrangle_zillow.get_zillow_data()
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,transactiondate
0,4.0,3.5,3100.0,1023282.0,1998.0,11013.72,6059.0,2017-01-01
1,2.0,1.0,1465.0,464000.0,1967.0,5672.48,6111.0,2017-01-01
2,3.0,2.0,1243.0,564778.0,1962.0,6488.30,6059.0,2017-01-01
3,4.0,3.0,2376.0,145143.0,1970.0,1777.51,6037.0,2017-01-01
4,3.0,3.0,1312.0,119407.0,1964.0,1533.89,6037.0,2017-01-01


In [4]:
df = df.replace(r'^\s*$', np.nan, regex=True)

In [5]:
df['area'] = df.calculatedfinishedsquarefeet

In [6]:
df = df.drop(columns='calculatedfinishedsquarefeet')

In [7]:
df = df.dropna()

In [10]:
col_list = ['bedroomcnt', 'bathroomcnt', 'area', 'taxvaluedollarcnt', 'taxamount']

In [11]:
df = wrangle_zillow.remove_outliers(df, 1, col_list)

In [12]:
df.shape

(45644, 8)

In [14]:
df['bed'] = df.bedroomcnt
df['bath'] = df.bathroomcnt
df['taxvalue'] = df.taxvaluedollarcnt

In [16]:
df = df.drop(columns=['bedroomcnt', 'bathroomcnt', 'taxvaluedollarcnt'])

In [18]:
df.head()

,yearbuilt,taxamount,fips,transactiondate,area,bed,bath,taxvalue
2,1962.0,6488.30,6059.0,2017-01-01,1243.0,3.0,2.0,564778.0
3,1970.0,1777.51,6037.0,2017-01-01,2376.0,4.0,3.0,145143.0
4,1964.0,1533.89,6037.0,2017-01-01,1312.0,3.0,3.0,119407.0
5,1982.0,3508.10,6111.0,2017-01-01,1492.0,3.0,2.0,331064.0
9,1980.0,4557.32,6037.0,2017-01-02,1290.0,2.0,3.0,371361.0


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45644 entries, 2 to 72240
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   yearbuilt        45644 non-null  float64
 1   taxamount        45644 non-null  float64
 2   fips             45644 non-null  float64
 3   transactiondate  45644 non-null  object 
 4   area             45644 non-null  float64
 5   bed              45644 non-null  float64
 6   bath             45644 non-null  float64
 7   taxvalue         45644 non-null  float64
dtypes: float64(7), object(1)
memory usage: 3.1+ MB


In [31]:
((df.transactiondate.str.contains('2017-05')) and (df.transactiondate.str.contains('2017-06')))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [37]:
# Selects units bought in the months of May - Aug 2017
df.loc[
         (df.transactiondate.str.contains('2017-05')) 
       | (df.transactiondate.str.contains('2017-06')) 
       | (df.transactiondate.str.contains('2017-07')) 
       | (df.transactiondate.str.contains('2017-08'))
        ]

,yearbuilt,taxamount,fips,transactiondate,area,bed,bath,taxvalue
578,1974.0,3490.16,6059.0,2017-08-04,1125.0,3.0,2.0,289445.0
950,1923.0,2627.48,6037.0,2017-07-21,1316.0,3.0,2.0,205123.0
1163,1970.0,2319.90,6037.0,2017-06-23,1458.0,3.0,2.0,136104.0
2494,1955.0,1399.27,6037.0,2017-08-23,1491.0,3.0,2.0,107110.0
3051,1949.0,7673.19,6037.0,2017-06-01,1650.0,3.0,2.0,614000.0
...,...,...,...,...,...,...,...,...
67820,1949.0,1450.15,6037.0,2017-08-31,900.0,2.0,2.0,119659.0
67821,1959.0,5552.68,6059.0,2017-08-31,1187.0,4.0,1.5,465999.0
67823,1964.0,6023.37,6037.0,2017-08-31,769.0,1.0,2.0,492395.0
67824,1969.0,7255.87,6037.0,2017-08-31,1212.0,2.0,2.0,600608.0


In [43]:
df.fips.value_counts()
# 6037 = Los Angeles county, 6059 = Orange county, ca, 6111 = Ventura County 

6037.0    27148
6059.0    14041
6111.0     4455
Name: fips, dtype: int64

In [42]:
df.isna().sum()

yearbuilt          0
taxamount          0
fips               0
transactiondate    0
area               0
bed                0
bath               0
taxvalue           0
dtype: int64

In [44]:
# Converting transactiondate into datetime

df['transactiondate'] = pd.to_datetime(df['transactiondate'],\
                        format = '%Y-%m-%d', errors = 'coerce')

# Creating columns for month, day, and week. We know they're all 2017
# so we don't need year

df['transactiondate_month'] = df['transactiondate'].dt.month
df['transactiondate_day'] = df['transactiondate'].dt.day
df['transactiondate_week'] = df['transactiondate'].dt.week

# Dropping transactiondate

df.drop(columns='transactiondate', inplace=True)

df.head()

,yearbuilt,taxamount,fips,area,bed,bath,taxvalue,transactiondate_month,transactiondate_day,transactiondate_week
2,1962.0,6488.30,6059.0,1243.0,3.0,2.0,564778.0,1,1,52
3,1970.0,1777.51,6037.0,2376.0,4.0,3.0,145143.0,1,1,52
4,1964.0,1533.89,6037.0,1312.0,3.0,3.0,119407.0,1,1,52
5,1982.0,3508.10,6111.0,1492.0,3.0,2.0,331064.0,1,1,52
9,1980.0,4557.32,6037.0,1290.0,2.0,3.0,371361.0,1,2,1


In [46]:
county_dict = {
                6059: "Orange",
                6037: "Los Angeles",
                6111: "Ventura"
                }

In [51]:
df.fips.dtypes

dtype('float64')

In [52]:
df['county'] = df.fips.replace(county_dict)

In [53]:
df.head()

,yearbuilt,taxamount,fips,area,bed,bath,taxvalue,transactiondate_month,transactiondate_day,transactiondate_week,county
2,1962.0,6488.30,6059.0,1243.0,3.0,2.0,564778.0,1,1,52,Orange
3,1970.0,1777.51,6037.0,2376.0,4.0,3.0,145143.0,1,1,52,Los Angeles
4,1964.0,1533.89,6037.0,1312.0,3.0,3.0,119407.0,1,1,52,Los Angeles
5,1982.0,3508.10,6111.0,1492.0,3.0,2.0,331064.0,1,1,52,Ventura
9,1980.0,4557.32,6037.0,1290.0,2.0,3.0,371361.0,1,2,1,Los Angeles
